In [102]:
from PIL import Image
import requests
from PIL import ImageFont
from PIL import ImageDraw
from PIL import ImageStat
import os
import random
import math
import time
import subprocess

In [39]:
def recommend_font_size(text):
    size = 135
    l = len(text)

    resize_heuristic = 0.9
    resize_actual = 0.985
    while l > 1:
        l = l * resize_heuristic
        size = size * resize_actual

    return int(size)

In [17]:
def wrap_text(text, w=30):
    new_text = ""
    new_sentence = ""
    for word in text.split(" "):
        delim = " " if new_sentence != "" else ""
        new_sentence = new_sentence + delim + word
        if len(new_sentence) > w:
            new_text += "\n" + new_sentence
            new_sentence = ""
    new_text += "\n" + new_sentence
    return new_text

In [24]:
def select_font():
    prefix = "fonts/"
    options = os.listdir(prefix)
    return prefix + random.choice(options)

In [165]:
def write_image(text, output_filename, background_img, IMAGE_WIDTH=1920, IMAGE_HEIGHT=1080, FONT='Roboto.ttf', FONT_SIZE=135, iF = ImageFont.truetype(FONT, FONT_SIZE), SPACING=0, FONT_COLOR=(255, 255, 255), BACKGROUND_COLOR=(0, 0, 0)):
    # setup
    text = wrap_text(text)
    img = Image.new("RGB", (IMAGE_WIDTH, IMAGE_HEIGHT), (255, 255, 255))

    # background
    back = background_img
    img_w, img_h = back.size
    bg_w, bg_h = img.size
    offset = (int((bg_w - img_w) / 3), int((bg_h - img_h) / 3))
    img.paste(back, offset)

    # text
    font = ImageFont.truetype(FONT, FONT_SIZE)
    draw = ImageDraw.Draw(img)
    img_w, img_h = img.size
    x = img_w / 2
    y = img_h / 2
    textsize = draw.multiline_textsize(text, font=iF, spacing=SPACING)
    text_w, text_h = textsize
    x -= text_w / 2
    y -= text_h / 2
    draw.multiline_text(align="center", xy=(x, y), text=text, fill=FONT_COLOR, font=font, spacing=SPACING)
    draw = ImageDraw.Draw(img)
    img = img.resize((960, 540), Image.ANTIALIAS)
    # output
    img.save(output_filename)
    return output_filename

In [166]:
def brightness( im_file ):
   im = im_file
   stat = ImageStat.Stat(im)
   r,g,b = stat.rms
   return math.sqrt(0.241*(r**2) + 0.691*(g**2) + 0.068*(b**2))

In [167]:
def finalImageProcess():
    im = Image.open(requests.get('https://picsum.photos/1920/1080/?blur=2', stream=True).raw)
    quote = requests.get("https://programming-quotes-api.herokuapp.com/Quotes/random").json()['en']
    time.sleep(1)
    bright = brightness(im)
    print(bright)
    FONT = select_font()
    FONT_SIZE = recommend_font_size(quote)
    IF = ImageFont.truetype(FONT, FONT_SIZE)
    COLOR = (255, 255, 255)
    if bright > 150:
        COLOR = (0, 0, 0)
    SPACING = 3
    print(write_image(quote, "img/quote.jpeg", im, IMAGE_WIDTH=1920, IMAGE_HEIGHT=1080, FONT=FONT, FONT_SIZE=FONT_SIZE, iF=IF, SPACING=SPACING, FONT_COLOR=COLOR, BACKGROUND_COLOR=(0, 0, 0)))
    time.sleep(5)
    subprocess.call("./push.sh", shell=True)

In [168]:
finalImageProcess()

129.64911522566865
img/quote.jpeg


In [92]:
BASE_URL = "https://graph.facebook.com/v13.0/app/uploads?access_token=" + ACCESS_TOKEN + "&file_length="+str(os.path.getsize("quote.jpeg"))+"&file_type=image/jpeg"

In [93]:
binary_data = open("quote.jpeg", "rb").read()

In [94]:
id = requests.post(BASE_URL, data=binary_data).json()['id']

In [95]:
id

'upload:MTphdHRhY2htZW50OmEwY2FkODBkLWVhNzItNDJjMi04YzQ0LTYyYWU2N2MyNjQ2Mz9maWxlX2xlbmd0aD0xNTM3MTkmZmlsZV90eXBlPWltYWdlJTJGanBlZw==?sig=ARYQM8EjWFi69BeiU-E'

In [96]:
file_handle = requests.post("https://graph.facebook.com/v13.0/" + id , headers = {'Authorization': 'OAuth ' + ACCESS_TOKEN, 'file_offset': '0', 'Content-Type': 'multipart/form-data'}, data=binary_data).json()['h']

In [97]:
file_handle

'4::aW1hZ2UvanBlZw==:ARaEgBWExn9tPo46Jgbb4Df-S2P3aT6UX4BFu0xlDeNk7zmwJKepr8XMQhl579IeONyemQz4k7sYmpsfCIqRyJ7cxdjXlxqcVK8Zkt_CFwV4LA:e:1651934544:1105434846699344:107965891688763:ARa0THy0d9tbyAV_y6E'

In [98]:
UPLOAD_STATUS = requests.get("https://graph.facebook.com/v13.0/" + id, headers={'Authorization': 'OAuth '+ ACCESS_TOKEN}).json()

In [100]:
os.path.getsize("quote.jpeg")

153719